## First Layer

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager

import datetime
import time
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import numpy as np
from datetime import date
date_today = date.today()

options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
options.add_argument("--disable-extensions")
driver = webdriver.Chrome(options=options, executable_path=r'/Users/ricardolu/Desktop/Design lab project/chromedriver')
driver.set_window_position(0, 0)
driver.set_window_size(1248, 768)

driver.get("https://www2.hm.com/en_us/men/new-arrivals/clothes.html")
# https://www2.hm.com/en_us/women/new-arrivals/clothes.html
time.sleep(1.5)

counter = 1
for i in range(0,6):
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollBy(0, 768);")

        # Wait to load page
        time.sleep(.2)

        # Calculate new scroll height and compare with last scroll height
        new_height = 768 * counter
        if new_height >= last_height:
            break

        counter += 1

    driver.execute_script("window.scrollBy(0, -368);")
    load_button = driver.find_element_by_xpath('/html/body/main/div[2]/div/div/div[2]/div[2]/button')
    load_button.click()

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollBy(0, 768);")

    # Wait to load page
    time.sleep(.3)

    # Calculate new scroll height and compare with last scroll height
    new_height = 768 * counter
    if new_height >= last_height:
        break

    counter += 1

soup = BeautifulSoup(driver.page_source,'html.parser')


In [2]:
product_name = []
original_price = []
sale_price = []
product_url = []
image_url = []
brand = []
product_type = []
timestamp = []

for i in soup.find_all('li',{'class':'product-item'}):
    if i.find('a',{'class':'link'}) != None:
        product_name.append((i.find('a',{'class':'link'})).text.strip())
    else:
        product_name.append(None)

    if i.find('span',{'class':'price club'}) != None:
        sale_price.append((i.find('span',{'class':'price club'})).text.strip().replace("Member price",""))
        if i.find('span',{'class':'price regular'}) != None:
            original_price.append((i.find('span',{'class':'price regular'})).text.strip())
    elif i.find('span',{'class':'price regular'}) != None:
        sale_price.append((i.find('span',{'class':'price regular'})).text.strip())
        original_price.append(None)
    else:
        sale_price.append(None)
        original_price.append(None)

    product_url.append("https://www2.hm.com"+i.find('a',{'class':'link'}).get('href'))

    if (i.find('div',{'class':'image-container'}).find('img',{'class':'item-image'})) !=None:
        image_url.append((i.find('div',{'class':'image-container'}).find('img',{'class':'item-image'})).get('data-src'))
    else:
        image_url.append(None)
    brand.append("hm")
    product_type.append("Men's Clothing")
    timestamp.append(date_today)

In [3]:
column_names = ["product_name", "original_price", "sale_price",'product_link', 'image_link']
df = pd.DataFrame(columns = column_names)

df['product_name'] = product_name
df['original_price'] = original_price
df['sale_price'] = sale_price
df['image_link'] = image_url
df['product_link'] = product_url

In [4]:
df

,product_name,original_price,sale_price,product_link,image_link
0,Embroidered-detail Sweater,None,$ 29.99,https://www2.hm.com/en_us/productpage.09440300...,//lp2.hm.com/hmgoepprod?set=source[/19/cb/19cb...
1,Slim Fit Henley Shirt,None,$ 12.99,https://www2.hm.com/en_us/productpage.09287750...,//lp2.hm.com/hmgoepprod?set=source[/11/a3/11a3...
2,Corduroy Shirt Jacket,None,$ 34.99,https://www2.hm.com/en_us/productpage.09616270...,//lp2.hm.com/hmgoepprod?set=source[/a0/a0/a0a0...
3,Relaxed-fit Hoodie,None,$ 24.99,https://www2.hm.com/en_us/productpage.06858140...,//lp2.hm.com/hmgoepprod?set=source[/1b/7a/1b7a...
4,Tapered Fit Sweatpants,None,$ 24.99,https://www2.hm.com/en_us/productpage.09048010...,//lp2.hm.com/hmgoepprod?set=source[/53/71/5371...
...,...,...,...,...,...
216,Premium Cotton Shirt,None,$ 39.99,https://www2.hm.com/en_us/productpage.09489080...,//lp2.hm.com/hmgoepprod?set=source[/78/6d/786d...
217,Slim Fit Twill Jacket,None,$ 149.00,https://www2.hm.com/en_us/productpage.09302100...,//lp2.hm.com/hmgoepprod?set=source[/dd/f5/ddf5...
218,Premium Cotton Shirt,None,$ 39.99,https://www2.hm.com/en_us/productpage.09489080...,//lp2.hm.com/hmgoepprod?set=source[/01/74/0174...
219,Swim Shorts,None,$ 14.99,https://www2.hm.com/en_us/productpage.09443700...,//lp2.hm.com/hmgoepprod?set=source[/e6/73/e673...


## Second Layer

In [5]:
product_description = []
material = []
picture_name = []
color = []
counter = 1
#driver = webdriver.Chrome(options=options, executable_path=r'/Users/ricardolu/Desktop/Design lab project/chromedriver')

for link in product_url:
    driver.get(link)
    time.sleep(1)

    soup = BeautifulSoup(driver.page_source,'html.parser')
    
    if soup.find('section',{'class':'name-price'}) != None:

        item_color = []
        if soup.find('ul',{'class':'inputlist clearfix'}).find('a',{'class':"filter-option miniature"}) != None:
            for color_i in soup.find('ul',{'class':'inputlist clearfix'}).find_all('a',{'class':"filter-option miniature"}):
                item_color.append(color_i.get("title"))
            color.append(item_color)
        else:
            color.append(None)

        if soup.find('p',{'class':'pdp-description-text'}) != None:
            product_description.append(soup.find('p',{'class':'pdp-description-text'}).text.strip())
        else:
            product_description.append(None)

        material_item = []
        if soup.find('div',{'class':'pdp-description-list-item'}) != None:
            for j in soup.find_all('div',{'class':'pdp-description-list-item'}):
                material_item.append(j.text.strip())
            material.append(material_item)
        else:
            material.append(None)
    else:
        color.append(None)
        product_description.append(None)
        material.append(None)

    picture_name.append('hm-men_' + str(counter) + '.jpg') # change variable
    counter += 1
driver.close()

In [6]:
# data cleaning for material
material_clean =[]
for i,v in enumerate(material):
    if v != None:
        item_material = []
        for j in v:
            if "%" in j:
                item_material.append(j)
        material_clean.append(item_material)
    else:
        material_clean.append(None)

In [7]:
# data cleaning for image url
picture_url = image_url.copy()
for i,v in enumerate(image_url):
    picture_url[i] = "https:" + v

In [8]:
column_names = ["brand", "product_type", "product_name", 'product_description', "price", 
                "sales_price", "color","material", "product_url", "picture_url", "timestamp", 'picture_name']
second_pass = pd.DataFrame(columns = column_names)
second_pass.brand = brand
second_pass.product_type = product_type
second_pass.product_name = product_name
second_pass.product_description = product_description
second_pass.price = original_price
second_pass.sales_price = sale_price
second_pass.color = color
second_pass.material = material_clean
second_pass.product_url = product_url
second_pass.picture_url = picture_url
second_pass.timestamp = timestamp
second_pass.picture_name = picture_name
second_pass

,brand,product_type,product_name,product_description,price,sales_price,color,material,product_url,picture_url,timestamp,picture_name
0,hm,Men's Clothing,Embroidered-detail Sweater,"Sweater in a soft, textured knit. Embroidered ...",None,$ 29.99,[Orange],[CompositionAcrylic 100%],https://www2.hm.com/en_us/productpage.09440300...,https://lp2.hm.com/hmgoepprod?set=source[/19/c...,2020-12-21,hm-men_1.jpg
1,hm,Men's Clothing,Slim Fit Henley Shirt,"Shirt in woven viscose fabric. Band collar, cl...",None,$ 12.99,None,[CompositionViscose 100%],https://www2.hm.com/en_us/productpage.09287750...,https://lp2.hm.com/hmgoepprod?set=source[/11/a...,2020-12-21,hm-men_2.jpg
2,hm,Men's Clothing,Corduroy Shirt Jacket,"Shirt jacket in soft cotton corduroy. Collar, ...",None,$ 34.99,[Turquoise],[CompositionCotton 100%],https://www2.hm.com/en_us/productpage.09616270...,https://lp2.hm.com/hmgoepprod?set=source[/a0/a...,2020-12-21,hm-men_3.jpg
3,hm,Men's Clothing,Relaxed-fit Hoodie,Relaxed-fit sweatshirt hoodie in cotton-blend ...,None,$ 24.99,"[Black, White, Beige, Burgundy, Light pink, Li...","[CompositionHood lining: Polyester 60%, Cotton...",https://www2.hm.com/en_us/productpage.06858140...,https://lp2.hm.com/hmgoepprod?set=source[/1b/7...,2020-12-21,hm-men_4.jpg
4,hm,Men's Clothing,Tapered Fit Sweatpants,Lightweight sweatpants in cotton-blend fabric....,None,$ 24.99,"[Black, Light gray melange, Gray melange]",[CompositionCotton 100%],https://www2.hm.com/en_us/productpage.09048010...,https://lp2.hm.com/hmgoepprod?set=source[/53/7...,2020-12-21,hm-men_5.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...
216,hm,Men's Clothing,Premium Cotton Shirt,"Shirt in premium cotton. Button-down collar, c...",None,$ 39.99,[White],[CompositionCotton 100%],https://www2.hm.com/en_us/productpage.09489080...,https://lp2.hm.com/hmgoepprod?set=source[/78/6...,2020-12-21,hm-men_217.jpg
217,hm,Men's Clothing,Slim Fit Twill Jacket,Conscious Exclusive. Jacket in Agraloop™ biofi...,None,$ 149.00,None,"[CompositionShell: Hemp 42%, Cotton 31%, Lyoce...",https://www2.hm.com/en_us/productpage.09302100...,https://lp2.hm.com/hmgoepprod?set=source[/dd/f...,2020-12-21,hm-men_218.jpg
218,hm,Men's Clothing,Premium Cotton Shirt,"Shirt in premium cotton. Button-down collar, c...",None,$ 39.99,None,[CompositionCotton 100%],https://www2.hm.com/en_us/productpage.09489080...,https://lp2.hm.com/hmgoepprod?set=source[/01/7...,2020-12-21,hm-men_219.jpg
219,hm,Men's Clothing,Swim Shorts,Swim shorts in woven fabric made from recycled...,None,$ 14.99,None,[CompositionShell: Polyamide 100%Lining: Polye...,https://www2.hm.com/en_us/productpage.09443700...,https://lp2.hm.com/hmgoepprod?set=source[/e6/7...,2020-12-21,hm-men_220.jpg


In [9]:
second_pass.to_csv("hm-men.csv", index=False, encoding = 'utf-8-sig')

In [10]:
import os
import requests as req

new_pd = pd.read_csv('hm-men.csv', encoding='utf-8-sig')

os.mkdir('hm-men')
i = 1
for index, link in enumerate(new_pd['picture_url']):
    img_data = req.get(link).content
    with open('hm-men/'+ "hm-men_"+str(index+1)+'.jpg','wb+') as f:
        f.write(img_data)
    i+=1